## Install TensorTrade

In [1]:
#!python -m pip install -e .. -U

## Setup Data Fetching




In [2]:
import pandas as pd

data = pd.read_csv('btc_data_train.csv')
test_data = pd.read_csv('btc_data_test.csv')

In [3]:
data.head()

,date,BTC:open,BTC:high,BTC:low,BTC:close,BTC:volume
0,7/1/2017 11:00,2505.56,2513.38,2495.12,2509.17,287000.32
1,7/1/2017 12:00,2509.17,2512.87,2484.99,2488.43,393142.50
2,7/1/2017 13:00,2488.43,2488.43,2454.40,2454.43,693254.01
3,7/1/2017 14:00,2454.43,2473.93,2450.83,2459.35,712864.80
4,7/1/2017 15:00,2459.35,2475.00,2450.00,2467.83,682105.41


## Create features with the data module

In [4]:
from tensortrade.data import Node, Module, DataFeed, Stream, Select


def rsi(price: Node, period: float):
    r = price.diff()
    upside = r.clamp_min(0).abs()
    downside = r.clamp_max(0).abs()
    rs = upside.ewm(alpha=1 / period).mean() / downside.ewm(alpha=1 / period).mean()
    return 100*(1 - (1 + rs) ** -1)


def macd(price: Node, fast: float, slow: float, signal: float) -> Node:
    fm = price.ewm(span=fast, adjust=False).mean()
    sm = price.ewm(span=slow, adjust=False).mean()
    md = fm - sm
    signal = md - md.ewm(span=signal, adjust=False).mean()
    return signal


features = []
for c in data.columns[1:]:
    s = Stream(list(data[c])).rename(data[c].name)
    features += [s]

btc_close = Select("BTC:close")(*features)

features += [
    rsi(btc_close, period=24).rename("BTC:rsi"),
    macd(btc_close, fast=10, slow=50, signal=5).rename("BTC:macd"),
]

feed = DataFeed(features)
feed.compile()



features_test = []
for c in test_data.columns[1:]:
    s = Stream(list(test_data[c])).rename(test_data[c].name)
    features_test += [s]

btc_close = Select("BTC:close")(*features_test)

features_test += [
    rsi(btc_close, period=24).rename("BTC:rsi"),
    macd(btc_close, fast=10, slow=50, signal=5).rename("BTC:macd"),
]

feed_test = DataFeed(features_test)
feed_test.compile()

In [5]:
feed.next()
feed.reset()

feed_test.next()
feed_test.reset()

C:\Users\suuser\PycharmProjects\tensortrade\tensortrade\data\stream\node.py:933: RuntimeWarning: invalid value encountered in double_scalars
  v = (w[::-1] * x).sum() / w.sum()


## Setup Trading Environment

In [6]:
from tensortrade.exchanges import Exchange
from tensortrade.exchanges.services.execution.simulated import execute_order
from tensortrade.data import Stream, DataFeed, Module
from tensortrade.instruments import USD, BTC, ETH
from tensortrade.wallets import Wallet, Portfolio
from tensortrade.environments import TradingEnvironment
from tensortrade.rewards import RiskAdjustedReturns
import copy

coinbase = Exchange("coinbase", service=execute_order)(
    Stream(list(data["BTC:close"])).rename("USD-BTC")
)

test_coinbase = Exchange("coinbase", service=execute_order)(
    Stream(list(test_data["BTC:close"])).rename("USD-BTC")
)

portfolio = Portfolio(USD, [
    Wallet(coinbase, 10000 * USD),
    Wallet(coinbase, 5 * BTC),
])

test_portfolio = Portfolio(USD, [
    Wallet(test_coinbase, 10000 * USD),
    Wallet(test_coinbase, 5 * BTC),
])

env = TradingEnvironment(
    feed=feed,
    portfolio=portfolio,
    use_internal=False,
    action_scheme="simple",
    reward_scheme=RiskAdjustedReturns(return_algorithm = 'sortino', window_size = 24),
    window_size=24
)

test_env = TradingEnvironment(
    feed=feed_test,
    portfolio=test_portfolio,
    use_internal=False,
    action_scheme="simple",
    reward_scheme=RiskAdjustedReturns(return_algorithm = 'sortino', window_size = 24),
    window_size=24
)


## Example Data Feed Observation

Even though this observation contains data from the internal data feed, since `use_internal=False` this data will not be provided as input to the observation history. The data that will be added to observation history of the environment will strictly be the nodes that have been included into the data feed that has been provided as a parameter to the trading environment.

In [7]:
env.feed.next()



{'BTC:open': 2505.56,
 'BTC:high': 2513.38,
 'BTC:low': 2495.12,
 'BTC:close': 2509.17,
 'BTC:volume': 287000.32,
 'BTC:rsi': nan,
 'BTC:macd': -1.0105496686365867e-13,
 'coinbase:/USD-BTC': 2509.17,
 'coinbase:/USD:/free': 10000.0,
 'coinbase:/USD:/locked': 0.0,
 'coinbase:/USD:/total': 10000.0,
 'coinbase:/BTC:/free': 5.0,
 'coinbase:/BTC:/locked': 0.0,
 'coinbase:/BTC:/total': 5.0,
 'coinbase:/BTC:/worth': 12545.85,
 'net_worth': 22545.85}

In [8]:
import tensortrade.agents.a2c_lstm_agent
from tensortrade.agents.a2c_lstm_agent import A2C_LSTM_Agent
import importlib
importlib.reload(tensortrade.agents.a2c_lstm_agent)

#agent = A2C_LSTM_Agent(env, validation_feed=feed_test)

#agent.train(n_steps=data.shape[0], batch_size=24*7, train_end=0.3, n_episodes=500, save_path="D:/Users/suuser/Desktop/test_results")



<module 'tensortrade.agents.a2c_lstm_agent' from 'C:\\Users\\suuser\\PycharmProjects\\tensortrade\\tensortrade\\agents\\a2c_lstm_agent.py'>

In [ ]:
import tensortrade.agents.a2c_agent
from tensortrade.agents.a2c_agent import A2CAgent
import importlib
importlib.reload(tensortrade.agents.a2c_agent)

env.set_feed(feed)
env.compile()
env.portfolio.reset()

agent = A2CAgent(env, test_env=test_env)
print('starting')
agent.train(n_steps=24*7*10, eps_end = 0.05, debug=False, discount_factor = 0.99, learning_rate=0.0001, batch_size=24*7, train_end=0.3, n_episodes=500, save_path="D:/Users/suuser/Desktop/a2c_test_results")




starting
====      AGENT ID: 0828865e-8893-4621-93c8-1f7d8e180265      ====
[10000.00 USD, 5.00000000 BTC]
====      TRAIN EPISODE ID (1/500): ed7888c4-f613-47d3-b766-fdb9c37db98e      ====
step 0/840
[10000.00 USD, 5.00000000 BTC]
None
exchange: 2509.17
step 24/840
[20286.20 USD, 0.64345846 BTC]
21862.415843616
exchange: 2449.6
step 48/840
[13000.14 USD, 3.40935514 BTC]
21531.369366822
exchange: 2502.3
step 72/840
[21875.89 USD, 0.00000000 BTC]
21875.89
exchange: 2614.93
step 96/840
[2422.65 USD, 7.37683202 BTC]
21396.304565361203
exchange: 2572.06
step 120/840
[8242.46 USD, 4.92530200 BTC]
21062.922599960002
exchange: 2602.98
step 144/840
[16862.54 USD, 1.42533919 BTC]
20505.70696964
exchange: 2556.0
step 168/840
[12126.52 USD, 3.18160865 BTC]
20106.7898963625
exchange: 2508.25
step 192/840
[13171.04 USD, 2.70013858 BTC]
20065.411841857203
exchange: 2553.34
step 216/840
[16025.87 USD, 1.31003865 BTC]
19294.259227112
exchange: 2494.88
step 240/840
[15762.49 USD, 1.09059247 BTC]
18336.

C:\Users\suuser\PycharmProjects\tensortrade\tensortrade\data\stream\node.py:414: RuntimeWarning: divide by zero encountered in true_divide
  return self.op(self.inputs[0].value, self.inputs[1].value)


step 24/840
[45476.43 USD, 0.00066379 BTC]
45481.1973331421
exchange: 7181.99
step 48/840
[45251.70 USD, 0.00001365 BTC]
45251.7944444865
exchange: 6919.01
step 72/840
[45251.78 USD, 0.00000136 BTC]
45251.789931318395
exchange: 7302.44
step 96/840
[45057.08 USD, 0.00000396 BTC]
45057.1091810024
exchange: 7368.94
step 120/840
[45057.08 USD, 0.00000396 BTC]
45057.1091969612
exchange: 7372.97
step 144/840
[44943.52 USD, 0.00000240 BTC]
44943.538935975994
exchange: 7889.99
step 168/840
[44920.37 USD, 0.00000306 BTC]
44920.3958481872
exchange: 8447.12
step 192/840
[44920.37 USD, 0.00000306 BTC]
44920.3943261432
exchange: 7949.72
step 216/840
[44444.86 USD, 0.00000000 BTC]
44444.86
exchange: 7814.61
step 240/840
[44175.03 USD, 0.00000055 BTC]
44175.0344938025
exchange: 8170.55
step 264/840
[43142.70 USD, 0.09683924 BTC]
43923.4818667784
exchange: 8062.66
step 288/840
[43921.55 USD, 0.00000254 BTC]
43921.570688477805
exchange: 8145.07
step 312/840
[43921.55 USD, 0.00000254 BTC]
43921.57100892

step 2520/840
[29691.46 USD, 0.00000253 BTC]
29691.477308995
exchange: 6841.5
step 2544/840
[29725.47 USD, 0.00000297 BTC]
29725.489530126
exchange: 6575.8
step 2568/840
[27623.82 USD, 0.30142981 BTC]
29757.1020656301
exchange: 7077.21
step 2592/840
[29396.46 USD, 0.00002793 BTC]
29396.6577092082
exchange: 7078.74
step 2616/840
[29284.40 USD, 0.00000266 BTC]
29284.4192456054
exchange: 7235.19
step 2640/840
[29276.56 USD, 0.00000516 BTC]
29276.5970178916
exchange: 7174.01
step 2664/840
[18526.11 USD, 1.52529481 BTC]
29037.5876654264
exchange: 6891.44
step 2688/840
[28997.13 USD, 0.00000410 BTC]
28997.158060072
exchange: 6843.92
step 2712/840
[28997.13 USD, 0.00000410 BTC]
28997.159219470002
exchange: 7126.7
step 2736/840
[28885.95 USD, 0.00030074 BTC]
28888.2046237208
exchange: 7496.92
step 2760/840
[28804.52 USD, 0.00000248 BTC]
28804.538542811202
exchange: 7476.94
step 2784/840
[28750.46 USD, 0.00000260 BTC]
28750.479692972
exchange: 7574.22
step 2808/840
[28246.92 USD, 0.00088584 BTC